In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statistics as stats
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import silhouette_score
from mlxtend.frequent_patterns import apriori
import warnings

warnings.filterwarnings('ignore')

In [ ]:
sheet1 = pd.read_excel('data.xlsx', 0)
sheet2 = pd.read_excel('data.xlsx', 1)

In [ ]:
db = pd.concat([sheet1, sheet2])

In [ ]:
db = pd.read_excel('data.xlsx')
db.to_csv('data.csv', index=False)

In [ ]:
db.head()

In [ ]:
db.tail()

In [ ]:
db.shape

In [ ]:
db.dtypes

In [ ]:
db.isnull().sum()

In [ ]:
db.duplicated()

In [ ]:
db.info()

In [ ]:
db.dropna(inplace=True)

In [ ]:
db.shape

In [ ]:
db.nunique()

In [ ]:
#There is no target varibale for clustering
#Dropped non values as there is a sufficient amount of data to work with

In [ ]:
first = [489434,12,2.95, 17530.0]
second = [489434,12,6.75, 13085.0]
total = 0
for i in range(len(first)):
    print((first[i]-second[i])**2)
    total += (first[i]-second[i])**2
total ** 0.5

Extremely far away due to the last feature.

In [ ]:
db.drop(['Customer ID'], axis=1)

In [ ]:
price = db['Price']
countries = db['Country']
label_encoder = LabelEncoder()
countries_encoded = label_encoder.fit_transform(countries) + 1 

In [ ]:
country_mapping = {country: i + 1 for i, country in enumerate(sorted(countries.unique()))}
countries_encoded = countries.map(country_mapping)

In [ ]:
db['countries_encoded'] = countries_encoded

In [ ]:
db.head()

In [ ]:
first = [489434,12,2.95, 35]
second = [489434,12,6.75, 35]
total = 0
for i in range(len(first)):
    print((first[i]-second[i])**2)
    total += (first[i]-second[i])**2
total ** 0.5

Points are now far closer.

## How many clusters needed?

In [ ]:
db_new = db.drop(['StockCode', 'Description', 'InvoiceDate', 'Country', 'Customer ID'], axis=1)

In [ ]:
kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)

In [ ]:
reshape = np.array([489434, 12, 2.95, 35]).reshape(-1, 1)  # Reshape into a 2D array with a single column

# Perform KMeans clustering
kmeans = KMeans(n_clusters=3,)  # Example: using 3 clusters
cluster_labels = kmeans.fit_predict(reshape)

print(cluster_labels)

In [ ]:
db_new

In [ ]:
db['Price'] = pd.to_numeric(db['Price'], errors='coerce')

In [ ]:
db_new = db_new[pd.to_numeric(db['Invoice'], errors='coerce').notnull()]

In [ ]:
db_new

In [ ]:
wcss = []
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    kmeans.fit(db_new)
    wcss.append(kmeans.inertia_)
print(wcss)

In [ ]:
plt.plot(range(2,10), wcss);

Biggest change is at 3, this means that 3 clusters would suit this dataframe best.

4 and 5 also hold an exaggerated bend.

In [ ]:
db_new_df = pd.DataFrame(data=db_new)

In [ ]:
scaler = StandardScaler()

In [ ]:
numeric_columns = db_new_df.select_dtypes(include='number').columns
db_new_df[numeric_columns] = scaler.fit_transform(db_new_df[numeric_columns])

In [ ]:
selected_columns = list(db_new_df.columns[1:6]) # Adjust column indices as needed
db_new2 = db_new_df[selected_columns]

In [ ]:
new_df = pd.DataFrame(data=db_new2, columns=selected_columns)

In [ ]:
new_df

In [ ]:
new_df.describe()

In [ ]:
y_pred = kmeans.fit_predict(new_df)

In [ ]:
kmeans.cluster_centers_

In [ ]:
y_pred==0

In [ ]:
new_df[y_pred==0]

In [ ]:
new_df.iloc[:,0]

In [ ]:
new_df

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)

# Fit the KMeans model to your data
kmeans.fit(new_df)

# Get the cluster centers
cluster_centers = kmeans.cluster_centers_

In [ ]:
def plotFeatures(col1, col2):
    plt.scatter(
        new_df[y_pred==0].iloc[:,col1],
        new_df[y_pred==0].iloc[:,col2],
        s=50,
        c='red'
    )
    plt.scatter(
        new_df[y_pred==1].iloc[:,col1],
        new_df[y_pred==1].iloc[:,col2],
        s=50,
        c='blue'
    )
    plt.scatter(
        new_df[y_pred==2].iloc[:,col1],
        new_df[y_pred==2].iloc[:,col2],
        c='green'
    )
    plt.scatter(
        kmeans.cluster_centers_[:,col1],
        kmeans.cluster_centers_[:,col2],
        s=200,
        c='yellow'
    )
    xmin, xmax = plt.xlim()  # Get the current limits of the x-axis
    plt.xlim(min(xmin, 0), xmax)
    
    plt.xlabel(x.columns[col1])
    plt.ylabel(x.columns[col2])
    plt.show();

In [ ]:
plotFeatures(2, 1)

In [ ]:
plotFeatures(0, 2)

In [ ]:
plotFeatures(0, 1)

## Silhouette Score

In [ ]:
silhouette_score(new_df, y_pred)

In [ ]:
for k in range(2, 10):
    kmeans = KMeans(n_clusters=3, random_state=42)
    y_pred = kmeans.fit_predict(new_df)
    print(f"The silhouette score for {3} clusters is: {silhouette_score(new_df, y_pred)}")

Excellant Silhouette score. Showing that the clusters are wll separated from each other as well as how similiar the points are within their own clusters are. The score is between -1 and +1. 0.92 is a great score.

## Market Basket Analysis (MBA)

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
db.head()

where to sell products:

In [ ]:
top_items = db['Description'].value_counts()
print(top_items)

In [ ]:
#pick top 5 items being bought

In [ ]:
te = TransactionEncoder()

In [ ]:
te_array = te.fit(db).transform(db)

In [ ]:
DB = pd.DataFrame(te_array, columns = te.columns_)

In [ ]:
DB.head()

In [ ]:
frequent_itemsets_ap = apriori(df, min_support = 0.01, use_colnames = True)